![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Introduction

This notebook demonstrations a method of using machine learning to discover candidate pairs of stocks for a pairs trading strategy. The process involves dimensionality reduction by principal component analysis, clustering with the OPTICS algorithm, and then several statistical tests.

# Step 1: Dimensionality Reduction

Before you can cluster assets into groups, we need to assign "coordinates" to each asset. The way we'll define the coordinates in this example is by gathering the daily returns of the assets, applying principal component analysis to reduce the matrix of daily returns to just 3 principal components, then finding how much each asset's series of returns contributes to each of the three principal components. The result effectively places each asset into a 3D space.

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.graph_objects as go

qb = QuantBook()

# Select a single trading day to use for the analysis.
end_date = datetime(2024, 1, 3)

# Get a list of all assets that were trading for the selected trading 
# day. 
universe_history = qb.universe_history(
    qb.universe.etf('IWB'), end_date - timedelta(1), end_date
)
symbols = [fundamental.symbol for fundamental in universe_history.iloc[0]]

# Get the historical daily returns of the selected assets.
prices = qb.history(
    symbols, end_date - timedelta(3 * 365), end_date, Resolution.DAILY
)['close'].unstack(0).dropna(axis=1)
daily_returns = prices.pct_change().dropna()

# Standardize each series of daily returns.
standardized_returns = StandardScaler().fit_transform(daily_returns)

# Perform PCA (reduce dataset dimensions)
pca = PCA(n_components=3, random_state=0)
pca.fit(standardized_returns)

factor_exposures = pd.DataFrame(
    index=[f"component_{i}" for i in range(pca.components_.shape[0])], 
    columns=daily_returns.columns,
    data=pca.components_ 
).T

# Display the results.
go.Figure(
    [
        go.Scatter3d(
            x=[row['component_0']],
            y=[row['component_1']],
            z=[row['component_2']],
            mode='markers',
            marker=dict(size=3, color='blue'),
            text=symbol,
            showlegend=False
        )
        for symbol, row in factor_exposures.iterrows()
    ],
    dict(
        scene=dict(
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z', 
            camera=dict(
                eye=dict(x=2, y=1.5, z=1.5)
            )
        ),
        title='3D Representation of Assets<br><sup>The coordinates represent '
            + 'the contribution the asset has to each component</sup>', 
    )
).show()

# Each row of pca.components_ corresponds to a principal component, and 
# the values within each row indicate the contribution of the original 
# features to that principal component.
factor_exposures

# Step 2: Clustering
Now that the assets are organized into 3D space, you can use a clustering algorithm to place them into groups. Let's use the OPTICS clustering algorithm.

In [2]:
from sklearn.cluster import OPTICS

clustering = OPTICS().fit(factor_exposures)

# Display the results.
group_by_cluster_id = {
    cluster_id: factor_exposures.iloc[
        [i for i, x in enumerate(clustering.labels_) if x == cluster_id]
    ]
    for cluster_id in sorted(set(clustering.labels_))
}
go.Figure(
    [
        go.Scatter3d(
            x=group['component_0'], y=group['component_1'], 
            z=group['component_2'], mode='markers',
            marker=dict(size=3), text=group.index,
            name=f"Group {cluster_id}" if cluster_id >= 0 else "Noisy group",
            visible=True if cluster_id >= 0 else 'legendonly'
        )
        for cluster_id, group in group_by_cluster_id.items()
    ],
    dict(
        scene=dict(
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z', 
            camera=dict(eye=dict(x=2, y=1.5, z=1.5))
        ),
        title='3D Representation of Assets<br><sup>The coordinates represent '
            + 'the contribution the asset has to each component</sup>', 
    )
).show()

# Drop noisy samples.
labels = clustering.labels_[clustering.labels_ != -1]
print(
    f"Out of {len(clustering.labels_)} assets, OPTICS found {len(labels)}",
    f"non-noisy samples and organzied them into {len(set(labels))} groups."
)

# Step 3: Absolute Rules of Disqualification
With the assets seperated into distinct groups, you can now look for candidate pairs within each group. In this example, a pair is selected if it complies with the following four conditions:
1. The pair’s constituents are cointegrated.
2. The pair’s spread Hurst exponent reveals a mean-reverting character.
3. The pair’s spread diverges and converges within convenient periods.
4. The pair’s spread reverts to the mean with enough frequency.

In [3]:
from itertools import combinations
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from hurst import compute_Hc
from arch.unitroot.cointegration import engle_granger

pairs_tested = 0
test_results_index = [
    'Failed at cointegration test', 'Failed at Hurst test', 
    'Failed at half-life test', 'Failed at crossing the mean'
]
test_results = pd.DataFrame(
    {"count" : [0] * len(test_results_index)}, 
    index=test_results_index
)
pairs_detected = []

# Loop through each cluster.
for cluster_id in range(len(set(labels))):
    # Select all assets in this cluster.
    cluster_symbols = list(
        factor_exposures.index[clustering.labels_ == cluster_id]
    )

    # Loop through all possible pairs in the cluster.
    for symbol_a, symbol_b in combinations(cluster_symbols, 2):
        pairs_tested += 1

        # Test 1: The pair’s constituents are cointegrated.
        # 1.1: Run the Engle-Granger tests.
        cointegration_test_results = [
            engle_granger(prices[symbol_a], prices[symbol_b]),
            engle_granger(prices[symbol_b], prices[symbol_a])
        ]

        # 1.2: Select the test result with the lowest test statistic 
        # (the more significant one).
        cointegration_test_result = sorted(
            cointegration_test_results, key=lambda x: x.stat
        )[0]

        # 1.3: Check the p-value of the Engle-Granger test.
        if cointegration_test_result.pvalue > 0.01:
            test_results.loc[test_results_index[0], 'count'] += 1
            continue  # Test failed: This pair is not cointegrated 

        # Test 2: The pair’s spread Hurst exponent reveals a 
        # mean-reverting character.
        spread = (
            prices[cointegration_test_result.cointegrating_vector.index[0]] 
            + cointegration_test_result.cointegrating_vector.values[1] 
            * prices[cointegration_test_result.cointegrating_vector.index[1]]
        )
        H, _, _ = compute_Hc(spread, kind='price', simplified=False)
        if H >= 0.5:
            test_results.loc[test_results_index[1], 'count'] += 1
            # Test failed: The spread doesn't lean towards 
            # mean-reverting.
            continue
        
        # Test 3: The pair's spread diverges and converges within 
        # convenient periods.
        lagged_spread = np.roll(spread, 1)
        lagged_spread[0] = 0
        spread_delta = spread - lagged_spread
        spread_delta.iloc[0] = 0
        # Run OLS regression to find regression coefficient.
        model = OLS(spread_delta, add_constant(lagged_spread))
        beta = model.fit().params.iloc[1]
        half_life = -np.log(2) / beta
        if not (1 < half_life < 252): 
            test_results.loc[test_results_index[2], 'count'] += 1
            # Test failed: Half life not between 1 day and 1 year.
            continue 

        # Test 4: The pair's spread reverts to the mean with enough 
        # frequency (12+ times per year).
        mean_value = spread.mean()
        crossings = (
            (spread > mean_value) & (spread.shift(1) <= mean_value) | 
            (spread < mean_value) & (spread.shift(1) >= mean_value)
        )
        num_crossings = crossings.sum()
        if num_crossings < 3 * 12:  # 36 months in 3 years
            test_results.loc[test_results_index[3], 'count'] += 1
            # Test failed: Spread didn't cross the mean at least 12 
            # times per year.
            continue

        pairs_detected.append(
            {
                'symbol_a': symbol_a, 
                'symbol_b': symbol_b, 
                'spread': spread, 
                'cointegrating_vector': cointegration_test_result.\
                    cointegrating_vector
            }
        )

test_results['percent_of_tested'] = round(
    test_results['count'] / pairs_tested, 4
)
test_results['cumulative_percent_of_tested'] = \
    test_results['percent_of_tested'].cumsum()

# Display Results

Let's analyse the search results. Print the number of pairs detected and show some plots for each candidate pair.

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print(f"Pairs tested: {pairs_tested}")
display(test_results) 
print(f"Pairs detected: {len(pairs_detected)}")

num_rows = len(pairs_detected)
fig = make_subplots(
    rows=num_rows, cols=2, 
    subplot_titles=num_rows * [
        "Prices of Candidate Pair", "Normalized Spread"
    ], 
    specs=[[{'secondary_y': True}, {}] for _ in range(num_rows)],
    vertical_spacing=0.02, horizontal_spacing=0.1
)

for i, pair in enumerate(pairs_detected):
    symbol_a = pair['symbol_a']
    symbol_b = pair['symbol_b']
    price_a = prices[symbol_a]
    price_b = prices[symbol_b]

    # Add traces to the left subplot with 2 y-axis labels
    fig.add_trace(
        go.Scatter(
            x=price_a.index, y=price_a, mode='lines', name=f'{symbol_a}', 
            line={'color': 'blue'}
        ),
        row=i+1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=price_b.index, y=price_b, mode='lines', name=f'{symbol_b}', 
            line={'color': 'red'}
        ),
        row=i+1, col=1,
        secondary_y=True
    )

    # Update y-axis titles for the left subplot
    for j in range(2):
        fig.update_xaxes(title_text="Time", row=i+1, col=j+1)
    fig.update_yaxes(
        title_text=f'Price({symbol_a})', row=i+1, col=1, secondary_y=False
    )
    fig.update_yaxes(
        title_text=f'Price({symbol_b})', row=i+1, col=1, secondary_y=True, 
        showgrid=False
    )
    fig.update_yaxes(
        title_text=f'Normalized Spread', row=i+1, col=2, secondary_y=False
    )

    normalized_spread = pd.Series(
        StandardScaler().fit_transform(
            pair['spread'].values.reshape(-1, 1)
        ).flatten(),
        index=pair['spread'].index
    )

    # Add trace to the right subplot
    fig.add_trace(
        go.Scatter(
            x=normalized_spread.index, y=normalized_spread, mode='lines', 
            name='Right Plot', line={'color': 'green'}
        ),
        row=i+1, col=2
    )

# Update layout
fig.update_layout(
    title_text="Pairs Detected<br><sup>Each row shows prices of the assets in "
        + "the pair and their normalized spread</sup>", 
    height=380*num_rows, showlegend=False
)

# Show the plot
fig.show()